# Set Up Database Connection

In [7]:
!pip install python-dotenv

In [8]:
import os
import psycopg2
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Get credentials from environment variables
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

# Print to verify (remove in production)
print(f"Connected to database: {DB_NAME} at {DB_HOST}")

# Connect to the database
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)

Connected to database: stock at 10.243.191.45


## Inserting data row-by-row into PostgreSQL database

*Bank tickers for TD, Scotiabank, and RBC - added by Mariia-Olena Zhupnyk and CIBC, BMO - added by Mariia Shekhovtsova*

In [25]:
%%capture
! pip install psycopg2 pandas

In [22]:
import os
import pandas as pd
import psycopg2

In [24]:
# Folder containing historical data CSV files
csv_folder = "D:\\CANADIAN_BANKS_STOCK_DATA"

# Bank tickers for TD, Scotiabank, and RBC
bank_tickers = {
    "TD": "TD.TO",
    "Scotiabank": "BNS.TO",
    "RBC": "RY.TO"
}

try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    cursor = conn.cursor()

    # SQL Query to Insert Data
    insert_query = """
    INSERT INTO stock_prices (stock_ticker, date, open_price, high_price, low_price, close_price, volume)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """

    # Process only historical data for TD, Scotiabank, and RBC
    for bank, ticker in bank_tickers.items():
        file_path = os.path.join(csv_folder, f"{bank}_historical_data.csv")

        if os.path.exists(file_path):  # Check if file exists
            df = pd.read_csv(file_path, skiprows=2)  # Skip first 2 rows

            # Rename columns to match database schema
            df.columns = ["date", "close_price", "high_price", "low_price", "open_price", "volume"]

            # Insert data row by row (without data type conversion)
            for _, row in df.iterrows():
                cursor.execute(insert_query, (ticker, row["date"], row["open_price"], row["high_price"],
                                              row["low_price"], row["close_price"], row["volume"]))

            conn.commit()
            print(f"✅ Data inserted successfully for {bank} ({ticker})")

    # Close database connection
    cursor.close()
    conn.close()

except Exception as e:
    print("❌ Error inserting data:", e)

✅ Data inserted successfully for TD (TD.TO)
✅ Data inserted successfully for Scotiabank (BNS.TO)
✅ Data inserted successfully for RBC (RY.TO)
